In [23]:
from Agent import Agent
from semantic_kernel.connectors.memory.chroma import ChromaMemoryStore
from chromadb.config import Settings
import os

agent_name = "OpenAI"
agent = Agent(agent_name=agent_name)
agent_config = agent.agent_config
embedder, chunk_size = await agent.get_embedder()
memories_dir = f"{os.getcwd()}/agents/{agent_name}/memories/"
chroma_client = ChromaMemoryStore(
    client_settings=Settings(
        chroma_db_impl="duckdb",
        persist_directory=memories_dir,
        anonymized_telemetry=False,
    ),
    persist_directory=memories_dir,
)

2023-05-31 09:09:51,853 | INFO | Running Chroma using direct local API.
2023-05-31 09:09:51,854 | WARNING | Using embedded DuckDB without persistence: data will be transient


In [24]:
await chroma_client.does_collection_exist_async("memories")


False

In [21]:
memories_exist = await chroma_client.does_collection_exist_async("memories")
if not memories_exist:
    await chroma_client.create_collection_async(collection_name="memories")
    memories = await chroma_client.get_collection_async(collection_name="memories")
else:
    memories = await chroma_client.get_collection_async(collection_name="memories")


2023-05-31 09:09:45,533 | INFO | Exiting: Cleaning up .chroma directory


In [7]:
from hashlib import sha256
from datetime import datetime
from semantic_kernel.memory.memory_record import MemoryRecord

description = "Test memory"
external_source_name = "Test memory"
content = "Test memory"

record = MemoryRecord(
    is_reference=False,
    id=sha256((content + datetime.now().isoformat()).encode()).hexdigest(),
    text=content,
    timestamp=datetime.now().isoformat(),
    description=description,
    external_source_name=external_source_name,  # URL or File path
    embedding=await embedder(content),
)

await chroma_client.upsert_async(
    collection_name="memories",
    record=record,
)

2023-05-31 08:58:33,272 | INFO | message='OpenAI API response' path=https://api.openai.com/v1/embeddings processing_ms=313 request_id=c599172dae99ca6fb4216ffd718eedfc response_code=200


'1b722caa2575a95c56336cd950d6fa694d5670b54834a3d3d9794a7d08d8642c'

In [8]:
results = await chroma_client.get_nearest_matches_async(
    collection_name="memories",
    embedding=await embedder(content),
    limit=1,
    min_relevance_score=0.1,
)

2023-05-31 08:58:35,371 | INFO | message='OpenAI API response' path=https://api.openai.com/v1/embeddings processing_ms=31 request_id=6f68dca8730eeb3c133357e46dfdce36 response_code=200


In [9]:
context = []
for memory, score in results:
    context.append(memory._text)
print(context)

['Test memory']


In [11]:
content = "Test memory"

await agent.memories.context_agent(query=content, top_results_num=2)

AttributeError: 'Memories' object has no attribute 'get_embedder'

In [2]:
# Store a memory
await memories.store_memory(content="Test memory", description="Test memory")

2023-05-31 08:15:38,537 | INFO | message='OpenAI API response' path=https://api.openai.com/v1/embeddings processing_ms=260 request_id=ed358be9efe6983fe6b50d7bdd795660 response_code=200


In [4]:
await memories.context_agent(query=content, top_results_num=2)

2023-05-31 08:16:04,802 | INFO | Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.
2023-05-31 08:16:04,803 | INFO | Running Chroma using direct local API.
2023-05-31 08:16:04,803 | WARNING | Using embedded DuckDB with persistence: data will be stored in: /home/josh/josh/Repos/AGiXT/agixt/agents/OpenAI/memories
2023-05-31 08:16:04,806 | INFO | No existing DB found in /home/josh/josh/Repos/AGiXT/agixt/agents/OpenAI/memories, skipping load
2023-05-31 08:16:04,806 | INFO | No existing DB found in /home/josh/josh/Repos/AGiXT/agixt/agents/OpenAI/memories, skipping load
2023-05-31 08:16:04,807 | INFO | Memories for OpenAI do not exist. Creating...
2023-05-31 08:16:05,417 | INFO | message='OpenAI API response' path=https://api.openai.com/v1/embeddings processing_ms=435 request_id=7e2931cce94d959f8849805ff50cfb62 response_code=200
2023-05-31 08:16:05,423 | INFO | Failed to get context: Index not found, please create an instance before querying


[]